In [1]:
# coding: utf-8
import warnings
warnings.filterwarnings('ignore')
import time
import os
import numpy as np
import keras
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Flatten, Conv2D, Conv1D, MaxPooling1D, MaxPooling2D,Activation
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec
from collections import *
import pandas as pd

Using TensorFlow backend.


In [2]:
#idenity paramaters
word_vec_len = 256

In [3]:
# get data
# get texts data
category2idx = {'AllTogether': 0, 'Baseball': 1, 'Boy-Girl': 2, 'C_chat':  3, 'CVS': 4,
                  'GameSale': 5, 'GetMarry': 6, 'Lifeismoney': 7, 'LoL': 8, 'MH': 9, 'MLB': 10, 'Mobilecomm': 11, 
                'movie': 12,'MuscleBeach':  13, 'NBA': 14,  'SENIORHIGH': 15, 'Stock': 16, 
                'Tennis': 17, 'Tos': 18, 'WomenTalk': 19}

train_df_sample = pd.read_pickle('train.pkl').sample(frac=1, random_state=123)

In [4]:
train_texts = train_df_sample.values
label_list = train_df_sample.label

test_pickle_df = pd.read_pickle('test.pkl')
test_texts = test_pickle_df["text"].values
# print(len(test_texts), test_texts[0]) 

In [5]:
train_texts_list = []
for text in train_texts:
    train_texts_list.append(text[0])
# print(len(train_texts_list), train_texts_list[0])

In [6]:
# get word embedding vector
answer = word2vec.Word2Vec.load("word2vec_20180430.model")
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)

In [7]:
print(len(wvv), len(wvv_keys_list))

87783 87783


In [8]:
texts_list = []
for text in train_texts_list:
#     print(text)
    texts_list.append(text)
    
for text in test_texts:
    texts_list.append(text)

In [9]:
print(type(texts_list), len(texts_list))
# print(label_list[0:10])

<class 'list'> 40000


In [10]:
# del word_vectors, wvv, train_texts_list

In [11]:
%%time
# word_index = tokenizer1.word_index
# word_counts = tokenizer1.word_counts
# print(len(word_index), word_index["巧合"], len(word_counts))
# num_words = max(max(sequences))
# print(num_words)

# word_index_keys_list = list(word_index.keys())
# word_index_values_list = list(word_index.values())
    

# docs_index = tokenizer1.texts_to_sequences(texts_list)
# print(num_words)


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [12]:
def text_to_index(corpus):
    new_corpus = []
    for doc in corpus:
        new_doc = []
        for word in doc:
            try:
                new_doc.append(word2idx[word])
            except:
                new_doc.append(0)
#         new_doc_arr = np.array(new_doc).reshape(1, max_doc_word_length)
        new_doc_arr = np.array(new_doc)
        new_corpus.append( new_doc_arr)
    return np.array(new_corpus)

In [13]:
embedding_matrix = np.zeros((len(wvv.items()) + 1, answer.vector_size))
word2idx = {}

vocab_list = [(word, word_vectors[word]) for word, _ in wvv.items()]
for i, vocab in enumerate(vocab_list):
    word, vec = vocab
    embedding_matrix[i + 1] = vec
    word2idx[word] = i + 1

In [14]:
# del answer
# print(embedding_matrix[1:10])

In [15]:
# # network model
print(embedding_matrix.shape, len(embedding_matrix[10]))
# (571339, 256) 256

(87784, 256) 256


In [16]:
# image_width = 250
# image_height = 3
# input_channel = 1

embedding_layer = Embedding(
                            input_dim= embedding_matrix.shape[0],
                            output_dim= 256,
                            weights=[embedding_matrix], 
                            input_length = 200,
                            trainable=False)
# model = getModel(embedding_layer, image_width, image_height, input_channel)
model = Sequential()
model.add(embedding_layer )
# model.add(Reshape(4, 2))
# conv2D = 
# model.add(Conv1D(64, 3, 
#             border_mode='same',
#             input_shape=(3, 256)))
# model.output_shape == (None, 64, 32, 32)

# model.add(Flatten())
model.add(Conv1D(256, 3,
#                 filters=(9), 
#                 kernel_size = (256),
                padding = 'same', 
#                  input_shape=(9, 32, 32),
#                  input_shape=(10, 32)
                )
         )
# model.add(MaxPooling1D(pool_size=2))    
model.add(Flatten())
model.add(Dense(2, activation='relu'))
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=0, verbose=0, mode='auto')
model.compile(optimizer='adam',loss='mae',  metrics=["accuracy"])
model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 256)          22472704  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 200, 256)          196864    
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102402    
Total params: 22,771,970
Trainable params: 299,266
Non-trainable params: 22,472,704
_________________________________________________________________


In [17]:
max_doc_word_length = 200
x_trains_texts = train_df_sample.text.append(test_pickle_df.text)
X_train_texts = text_to_index(x_trains_texts)
X_train = pad_sequences(X_train_texts, maxlen= max_doc_word_length)

In [18]:
# sequences2 = np.reshape(sequences1,(2, 20000, 200)) 
# print(sequences1.shape, label_list.shape, label_list[0])

Y_label_list = np.zeros((36000, 2))
for ids in range(0, 36000):
    Y_label_list[ids][0] = label_list[ids][0]
    Y_label_list[ids][1] = label_list[ids][1]
print(Y_label_list.shape)

(36000, 2)


In [19]:
# print(len(X_train))

In [ ]:
%%time
history = model.fit(x = X_train[0:36000], 
                    y = Y_label_list, 
#                     validation_split=0.1, 
                    batch_size= 1800,
                    epochs = 70, verbose = 1)

Epoch 1/70
36000/36000 [==============================] - 104s 3ms/step - loss: 11.8513 - acc: 0.9561
Epoch 2/70
36000/36000 [==============================] - 103s 3ms/step - loss: 11.5722 - acc: 0.9705
Epoch 3/70
 7200/36000 [=====>........................] - ETA: 1:26 - loss: 12.0156 - acc: 0.9721

In [ ]:
import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
# import numpy as np

np_loss_history = np.array(loss_history)
np.savetxt("loss_history.txt", np_loss_history, delimiter=",")

fig = plt.figure()
ax = plt.axes()
x = np.linspace(0, 1, 50)
loss_history = np.loadtxt("loss_history.txt")
plt.plot(x, loss_history, '-g');  # dotted red


In [ ]:
model.save('my_model.h5') 

In [ ]:
# evaluate the model
loss_accuracy = model.evaluate(X_train[0:100], Y_label_list[0:100], verbose=1)
print(type(loss_accuracy), loss_accuracy)

test_sequences1 = X_train[36000:40000]

predict_res = model.predict(test_sequences1, batch_size= 3600, verbose=0)
# print(len(predict_res), predict_res)

final_res = []
for pre_res in predict_res:
    final_res.append(pre_res)
print(final_res)

In [ ]:
print(predict_res[0])

In [ ]:
# result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
print(len(final_res))
result_txt = "local_result001" + ".txt"
ids = 0
with open(result_txt, 'w') as out:
    out.write("id,good,bad" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(int (round(value[0]))) + "," + str(int (round(value[0]))) + '\n')
        ids += 1